In [12]:
# Imports
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship


In [13]:
# Grab a reference to the current directory
rootdir = os.getcwd()


In [14]:
# Use `os.scandir` to get a list of all files in the current directory
csvs = os.scandir(rootdir)


In [15]:
# Iterate through the list and clean/process any CSV file using Pandas
for csv in csvs:
    # Only open CSV file extensions
    if csv.name.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(csv.path, dtype=object)

        # Drop the location column since lat, lon, and elev already exist
        if 'location' in df.columns:
            df = df.drop(['location'], axis=1).reset_index(drop=True)
        
        # Use the mean to fill in any NaNs
        df.fillna(df.mean(), inplace=True)

        # Save the cleaned files with a `clean_` prefix
        df.to_csv(os.path.join(rootdir, f"clean_{csv.name}"), index=False)


In [25]:
# Verify that the cleaned files were created
!ls

app.py
clean_clean_hawaii_measurements.csv
clean_clean_hawaii_stations.csv
clean_hawaii_measurements.csv
clean_hawaii_stations.csv
climate_analysis.ipynb
datanotebook.ipynb
hawaii.sqlite
hawaii_measurements.csv
hawaii_stations.csv
Instructions
README.md


In [17]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column

# Declare a Base object here
Base = declarative_base()


In [18]:
# Define the ORM class for `Measurements`
class Measurement(Base):
    
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"


In [19]:
# Define the ORM class for `Station`
class Station(Base):
    
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"


In [20]:
# Use `create_all` to create the tables
Base.metadata.create_all(engine)


NameError: name 'engine' is not defined

In [21]:
# Verify that the table names exist in the database
engine.table_names()


NameError: name 'engine' is not defined

In [22]:
# Use Pandas to Bulk insert each CSV file into their appropriate table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, Station.__table__, 'clean_hawaii_stations.csv')
populate_table(engine, Measurement.__table__, 'clean_hawaii_measurements.csv')


NameError: name 'engine' is not defined

In [23]:
# Use a basic query to validate that the data was inserted correctly for table `measurement`
engine.execute("SELECT * FROM measurement LIMIT 1").fetchall()


NameError: name 'engine' is not defined

In [24]:
# Use a basic query to validate that the data was inserted correctly for table `station`
engine.execute("SELECT * FROM station LIMIT 1").fetchall()


NameError: name 'engine' is not defined